# <center>Network Science</center>

## <center>Home Assignment #3: Centralities and Assortativity</center>

### <center>Student: Ruslan Kireev</center>

#### <hr /> General Information

**Due Date:** 13.03.2016 23:59 <br \>
**Late submission policy:** -0.2 points per day <br \>


Please send your reports to <network.hse.2016@gmail.com> with message subject of the following structure:<br \> **[HSE Networks 2015] *{LastName}* *{First Name}* HA*{Number}***

Support your computations with figures and comments. <br \>
If you are using IPython Notebook you may use this file as a starting point of your report.<br \>
<br \>
<hr \>

## Problems

### Task 1

Compute degree centrality, Pagerank and  HubAuthorities scores for the [flickr](https://www.hse.ru/data/2016/02/24/1139618969/flickr.mat) network. 

Data contains sparse matrix A and list of user names.
This is a “denser” part of the Flickr photo sharing site friendship graph from 2006. Edge direction corresponds to friendship requests (following). Some of the links are reciprocal,others not.  

Provide top 50 names in each ranking, compare results

You can load .mat files with the following commands:

----------------------------

The first thing should be done is to represent our .mat data as a networkx graph:

In [1]:
import scipy.io
import networkx as nx

data = scipy.io.loadmat('flickr.mat')
G = nx.from_scipy_sparse_matrix(data['A'],create_using=nx.DiGraph())
mapping = dict(zip(G.nodes(),data['names']))
G = nx.relabel_nodes(G,mapping)

Calculate each ranking and sort it out:

In [2]:
degree_sorted = sorted(nx.degree_centrality(G).items(), key=lambda (k,v): v, reverse=True)
degree_in_sorted = sorted(nx.in_degree_centrality(G).items(), key=lambda (k,v): v, reverse=True)
degree_out_sorted = sorted(nx.out_degree_centrality(G).items(), key=lambda (k,v): v, reverse=True)
pagerank_sorted = sorted(nx.pagerank(G).items(), key=lambda (k,v): v, reverse=True)
hubs, authorities = nx.hits(G)
hubs_sorted = sorted(hubs.items(), key=lambda (k,v): v, reverse=True)
authorities_sorted = sorted(authorities.items(), key=lambda (k,v): v, reverse=True)

Find top 50 names and print kind of line by line

In [3]:
degree_top50 = []
degree_in_top50 = []
degree_out_top50 = []
pagerank_top50 = []
hubs_top50 = []
authorities_top50 = []
for i in xrange(50):
    degree_top50.append(degree_sorted[i][0].strip())
    degree_in_top50.append(degree_in_sorted[i][0].strip())
    degree_out_top50.append(degree_out_sorted[i][0].strip())
    pagerank_top50.append(pagerank_sorted[i][0].strip())
    hubs_top50.append(hubs_sorted[i][0].strip())
    authorities_top50.append(authorities_sorted[i][0].strip())

In [4]:
import numpy as np
a = np.array((degree_top50,degree_in_top50,pagerank_top50,authorities_top50,degree_out_top50,hubs_top50))
print np.transpose(a)

[[u'awfulsara' u'awfulsara' u'awfulsara' u'awfulsara' u'anildash'
  u'mrpiink']
 [u'drp' u'drp' u'drp' u'drp' u'tozzer' u'automat']
 [u'DrJoanne' u'*Ivan*' u'antimethod' u'DrJoanne' u'AtiRanA' u'schizoo23']
 [u'antimethod' u'antimethod' u'BombDog' u'*Ivan*' u'pixietart'
  u'lorrainemd']
 [u'*Ivan*' u'DrJoanne' u'*Ivan*' u'antimethod' u'jakedobkin' u'sgoralnick']
 [u'BombDog' u'BombDog' u'MaD Gi\xc2\xaeL\xe2\x80\xa2\xe2\x84\xa2'
  u'BombDog' u'Buntekuh' u'*starlet*']
 [u'deborah lattimore' u'Simon Pais' u'DrJoanne' u'Simon Pais'
  u'Jakes_World' u'brynfoto']
 [u'Simon Pais' u'deborah lattimore' u'Simon Pais' u'deborah lattimore'
  u'brainware3000' u'liquid pixel']
 [u'cymagen' u'MaD Gi\xc2\xaeL\xe2\x80\xa2\xe2\x84\xa2'
  u'deborah lattimore' u'cymagen' u'maximolly' u'noahstone']
 [u'notraces' u'notraces' u'cymagen' u':Nikola' u'Andreia Lopes'
  u'isherwood']
 [u'MaD Gi\xc2\xaeL\xe2\x80\xa2\xe2\x84\xa2' u':Nikola' u':Nikola'
  u'lorrainemd' u'inthegan' u'Arno-MM']
 [u'aleyna' u'cymagen' 

As we can see from above there are quite simillar names on the top for all rankings except out-degree centrality and hubs. To get more formal let's calculate intersections for most interesting combinations:

In [5]:
print len(set(degree_top50) & set(pagerank_top50) & set(authorities_top50))
print len(set(degree_out_top50) & set(hubs_top50))
print len(set(degree_top50) & set(pagerank_top50) & set(authorities_top50) & set(hubs_top50))

30
6
10


We can conclude, that degree centrality, pagarank and authorities show quite alike results(60% of names are common). Besides, as we know, good hubs point to good authorities, that's why it's reasonable to compare outdegree centrality and hubs. It was shown to be only 6 out of 50 names, while even an intersection with all the others gives 10 common names. 

### <hr /> Task 2

Here are the [Facebook friendship graphs](https://www.hse.ru/data/2016/03/05/1125656999/universities.zip) from several US universities from 2005 (one year after fb launch).

Data contains a A matrix (sparse) and a "local_info" variable, one row per node: 
a student/faculty status flag, gender, major, second major/minor (if applicable), dorm/house, year, and high school. 
Missing data is coded 0.

Compute node degree assortativity (mixining by node degree) and assortativity coefficient (modularity) for gender, major, dormitory, year, high school for all universities and compare the results.

In [6]:
universities = ['Berkeley13.mat','Caltech36.mat','Harvard1.mat','Texas80.mat','Princeton12.mat','Oklahoma97.mat']
data = []
for x in universities:
    data.append(scipy.io.loadmat(x))

This function takes a dataset and an attribute and returns assortativity for given attribute and degree assortativity if there are no attribute. 

In [7]:
def assortativity(data, attr=None):
    attrs = dict([('gender',1), ('major',2), ('dormitory',4), ('year',5), ('high_school',6)])
    G = nx.from_scipy_sparse_matrix(data['A'],create_using=nx.DiGraph())
    if not attr:
        return nx.degree_assortativity_coefficient(G)
    mapping = dict(zip(G.nodes(),data['local_info'][:,attrs[attr]]))
    nx.set_node_attributes(G,attr, mapping)
    return nx.attribute_assortativity_coefficient(G,attr)

In [8]:
for d in data:
    print assortativity(d)

0.0103134590889
-0.0652729511948
0.145054887671
0.163892412814
0.091092123568
0.0736732579392


Degree assortativity coefficient shows, that nodes tend to be connected with other nodes with similar degree values (closer to 1 -- denser the core of high degree nodes). 

So we can point out 2 universities with highest values: Harvard and Texas.

Negative coefficient shows that Caltech has rather disassortative network.

In [9]:
i=0
for d in data:
    print universities[i]
    i+=1
    for attr in ['gender','major','dormitory','year','high_school']:
        print attr, assortativity(d,attr)

Berkeley13.mat
gender 0.0409228376393
major 0.058255029946
dormitory 0.14727644076
year 0.308121015806
high_school 0.0773753653543
Caltech36.mat
gender 0.0463314364559
major 0.0347194237236
dormitory 0.349153100873
year 0.237531904813
high_school 0.00266977930658
Harvard1.mat
gender 0.0452699773794
major 0.0496769283286
dormitory 0.12567663505
year 0.421159642116
high_school 0.0239867253741
Texas80.mat
gender 0.0796930681412
major 0.0597127278334
dormitory 0.204837604339
year 0.246194684162
high_school 0.147992848231
Princeton12.mat
gender 0.0506107399953
major 0.0543901188714
dormitory 0.0946784195542
year 0.446744620584
high_school 0.018783655245
Oklahoma97.mat
gender 0.0928372959827
major 0.0363790887617
dormitory 0.280335243729
year 0.20116469147
high_school 0.139468406924


First thing to note -- small values for gender and major attributes in each university. That is, there are no cores of mostly males and mostly females, neither cores of majors. 
Secondly, in Texas and Oklahoma high school cores are more emphasized, than in the rest.
Finally, in general for all datasets dormitory (Oklahoma, Caltech) and year(Berkeley, Harvard, Texas, Princeton) are most significant attributes. So people of the same year are more connected. 